# Joint Fitting example

This notebook covers how to perform joint inference and optimization for multiple observations, using ndspec's fit objects. This notebook has combines two analyses from previous tutorials and explores how constraints on parameters change when performing joint fitting and inference. See the following for more detailed explanations of fitting: [power spectra](https://ndspec.readthedocs.io/en/latest/fit_psd.html), [time-averaged spectra](https://ndspec.readthedocs.io/en/latest/fit_spec.html), [1-D cross-spectra](https://ndspec.readthedocs.io/en/latest/fit_cross_1d.html) and [2-D cross-spectra](https://ndspec.readthedocs.io/en/latest/fit_cross_2d.html).

First, we'll import important packages (as well as set some specific environment variables that helps with performance when using numpy in parallel frameworks such as in emcee which is further explained in [performance considerations](https://ndspec.readthedocs.io/en/latest/)).

In [ ]:
import os

os.environ["OMP_NUM_THREADS"] = "1" 
os.environ["OPENBLAS_NUM_THREADS"] = "1" 
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 

import numpy as np

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import matplotlib.pyplot as plt

from ndspec.Response import ResponseMatrix
import ndspec.FitCrossSpectrum as fitcross
import ndspec.FitTimeAvgSpectrum as fitspec
import ndspec.models as models
import ndspec.XspecInterface as XSModels

from lmfit import Model as LM_Model

### Loading Data
We'll be using the same data and performing the same data preparation as the earlier tutorials: [Fitting a time averaged spectrum](https://ndspec.readthedocs.io/en/latest/fit_spec.html) and [Fitting a one-dimensional cross-spectrum](https://ndspec.readthedocs.io/en/latest/fit_cross_1d.html). We'll load all of our data, and rebin the energy matrices into more appropriate bins for the cross-spectrum and time-averaged energy spectrum. See the previous tutorials for the full explanation of why and how we rebin the response matrices.

In [ ]:
#load and rebin response - we need this to have the exact intervals for our energy bins
path = os.getcwd()

rmfpath = path+"/data/1200120106_rmf.pha"
nicer_matrix = ResponseMatrix(rmfpath)
arfpath = path+"/data/1200120106_arf.pha"
nicer_matrix.load_arf(arfpath)

#deal with cross-spectrum
nicer_energy_rebin = nicer_matrix.rebin_energies(10)

rebin_bounds = np.geomspace(0.5,10,41)
rebin_bounds = np.append(np.min(nicer_matrix.emin),rebin_bounds)
rebin_bounds = np.append(rebin_bounds,np.max(nicer_matrix.emax))

rebin_bounds_lo = rebin_bounds[:-1]
rebin_bounds_hi = rebin_bounds[1:]
rebin_response = nicer_matrix.rebin_channels(rebin_bounds_lo,rebin_bounds_hi)

channel_grid = 0.5*(rebin_response.emax+rebin_response.emin)
channel_width = rebin_response.emax-rebin_response.emin
#finally, find the array of energy channel edges that line up with the response and are roughly geometrically spaced
fine_channel_grid_edges = np.append(channel_grid-0.5*channel_width,channel_grid[-1]+0.5*channel_width[-1])

#deal with energy spectrum
rebin_matrix = nicer_matrix.rebin_energies(4)
spectrum_fit = fitspec.FitTimeAvgSpectrum()

spectrum_fit.set_data(rebin_matrix,os.getcwd()+'/data/1200120106_rebin.pha')

In [ ]:
#convert from lightcurves to lag-energy spectra with Stingray
from ndspec.SimpleFit import load_lc
from stingray.fourier import avg_cs_from_timeseries, avg_pds_from_timeseries
from stingray.fourier import poisson_level
from stingray.fourier import error_on_averaged_cross_spectrum
from stingray.utils import show_progress

def lag_from_lcs(lc_strings,lc_ref,freq_lo,freq_hi,seg_size,time_res):
    time_ref,counts_ref,gtis_ref = load_lc(lc_ref)
    results = avg_pds_from_timeseries(
            time_ref,
            gtis_ref,
            seg_size,
            time_res,
            silent=True,
            norm="none",
            fluxes=counts_ref,)
    freq = results["freq"]
    ref_power = results["power"]
    m_ave = results.meta["m"]
    ref_power_noise = poisson_level(norm="none", n_ph=np.sum(counts_ref) / m_ave)
    freq_mask = (freq>freq_lo) & (freq<freq_hi)
    n_freqs = freq_mask[freq_mask==True].size
    mean_ref_power = np.mean(ref_power[freq_mask])
    m_tot = n_freqs * m_ave
    f_mean = (freq_lo + freq_hi)*0.5

    lag_spec = []
    lag_spec_err = []

    for i in range(len(lc_strings)):
        time_sub,counts_sub,gtis_sub = load_lc(lc_strings[i])

        results_cross = avg_cs_from_timeseries(
                time_sub,
                time_ref,
                gtis_sub,
                seg_size,
                time_res,
                silent=True,
                norm="none",
                fluxes1=counts_sub,
                fluxes2=counts_ref,
            )

        results_ps = avg_pds_from_timeseries(
                time_sub,
                gtis_sub,
                seg_size,
                time_res,
                silent=True,
                norm="none",
                fluxes=counts_sub,
            )

        sub_power_noise = poisson_level(
            norm="none", n_ph=np.sum(counts_sub) / results_ps.meta["m"]
        )
        cross = results_cross["power"]
        sub_power = results_ps["power"]
        Cmean = np.mean(cross[freq_mask])
        mean_sub_power = np.mean(sub_power[freq_mask])

        _, _, phi_e, _ = error_on_averaged_cross_spectrum(
                Cmean,
                mean_sub_power,
                mean_ref_power,
                m_tot,
                sub_power_noise,
                ref_power_noise,
                common_ref=True,
            )

        phase = np.angle(Cmean)
        lag_spec = np.append(lag_spec,phase/(2*np.pi*f_mean))
        lag_spec_err = np.append(lag_spec_err,phi_e/(2*np.pi*f_mean))



    return lag_spec, lag_spec_err

In [ ]:
#round the bins of the channels in each lightcurve
fine_channel_string = []
for i in range(len(fine_channel_grid_edges)):
    bin = "%.0f" % round(100.*fine_channel_grid_edges[i],2)
    fine_channel_string = np.append(fine_channel_string,bin)
fine_channel_string = np.array(fine_channel_string,dtype=int)

#create the path to each lightcurve
fine_full_string = []
for i in range(len(fine_channel_string)-1):
    bin_string = "_"+str(fine_channel_string[i])+"-"+str(fine_channel_string[i+1]-1)
    lc_string = path+"/data/lightcurves/ni1200120106mpu7_sr" + bin_string + ".lc"
    fine_full_string = np.append(fine_full_string,lc_string)

ref_path = path+"/data/lightcurves/ni1200120106mpu7_sr_reference.lc"

#define the frequency bounds and, time resolution, reference band  and segment size, and load the data
freqs = np.geomspace(0.2,16,7)

lags = []
lags_err = []

dt = 0.03
segment_size = 5
ref_band = [0.5, 10]

for i in show_progress(range(6)):
    lag,lag_err=lag_from_lcs(fine_full_string,ref_path,freqs[i],freqs[i+1],segment_size,dt)
    lags = np.append(lags,lag)
    lags_err = np.append(lags_err,lag_err)

In [ ]:
lags_fit = fitcross.FitCrossSpectrum()
lags_fit.set_coordinates("lags")
lags_fit.set_product_dependence("energy")

#nicer_matrix
#nicer_energy_rebin
lags_fit.set_data(nicer_energy_rebin,ref_band,fine_channel_grid_edges,
                  lags,lags_err,
                  freq_bins=freqs,
                  time_res=dt,seg_size=segment_size)
lags_fit.ignore_energies(0,0.5)
lags_fit.ignore_energies(10.0,fine_channel_grid_edges[-1])

### Define models
So far, we've loaded in the data and created two Fit objects, each with their own data. We now want to model the data in the same way as [Fitting a time averaged spectrum](https://ndspec.readthedocs.io/en/latest/fit_spec.html) and [Fitting a one-dimensional cross-spectrum](https://ndspec.readthedocs.io/en/latest/fit_cross_1d.html), but instead jointly.

First, we have to define our time-averaged spectrum and time-lags models using a mix of ndspec's phemonological models as well as xspec models that have been loaded by ndspec's xspec interface.

In [ ]:
model_library = XSModels.FortranInterface()

def tbabs(ear, params):
    pass

def diskbb(ear, params):
    pass

model_library.load_models({
    "diskbb": diskbb,
    "tbabs": tbabs
})

# Define the models for the time-averaged spectrum
def powerlaw(energ,index,norm_pl):
    par_array = np.array([norm_pl,index])
    model = models.powerlaw(energ,par_array)
    if np.any(np.isnan(model)):
        print("Bad PL")
    return model

def wrap_diskbb(energ,kT,norm_disk):
    par_array = np.array([kT,norm_disk])
    model = model_library.diskbb(energ,par_array)
    if np.any(np.isnan(model)):
        print("Bad disk")
    return model

#the xspec wrappers completely fuck up with emcee
def blackbody(energ,kT,norm_disk):
    par_array = np.array([norm_disk,kT])
    model = models.bbody(energ,par_array)
    
    if np.any(np.isnan(model)):
        print("Bad BB")
    return model

def wrap_tbabs(energ,nH):
    par_array = np.array([nH])
    model = model_library.tbabs(energ,par_array)
    if np.any(np.isnan(model)):
        print("Bad tbabs")
    return model

#Define the models for the time-lag cross-spectrum
from ndspec.Timing import PowerSpectrum, CrossSpectrum

def wrap_tbabs_tl(energs,nH):
    par_array = np.array([nH])
    model = model_library.tbabs(energs,par_array)
    if np.any(np.isnan(model)):
        print("Bad tbabs")
    return model

#this is defined in the Fourier domain as a transfer function
def pivoting_lowecut(energs,freqs,norm_pl2,index,gamma_0,gamma_slope,phi_0,phi_slope,nu_0,cutoff):
    param_array = np.array([norm_pl2,index,gamma_0,gamma_slope,phi_0,phi_slope,nu_0])
    model = np.transpose(models.pivoting_pl(freqs,energs,param_array))*np.exp(-cutoff/energs)
    return model

#the reverberation models are defined in the time domain, so we need to define a cross spectrum to
#convert it to a transfer function to then return
def reverb(energs,times,rev_norm,rev_temp,rise_slope,decay_slope,peak_time,temp_slope):
    param_array = np.array([rev_norm,rev_temp,rise_slope,decay_slope,peak_time,temp_slope])
    impulse_response = models.bbody_bkn(times,energs,param_array)
    cross_spec = CrossSpectrum(times,energ=energs)
    cross_spec.set_impulse(impulse_response)
    cross_spec.transfer_from_irf()
    model = np.transpose(cross_spec.trans_func)
    return model

psd = PowerSpectrum(lags_fit._times)
#set Lorentzian parameters from the PSD fit
l1_pars = np.array([0.056,0.31,0.251])
l2_pars = np.array([0.85,0.09,0.164])
l3_pars = np.array([2.52,0.35,0.117])
psd_model = models.lorentz(psd.freqs,l1_pars)+models.lorentz(psd.freqs,l2_pars)+models.lorentz(psd.freqs,l3_pars)
psd.power_spec = psd_model

Then, we define the actual composite models now and set our _individual_ FitObjects to use those models.

In [ ]:
# Set the models for the time-averaged spectrum
# the unfolded model is a composite of tbabs, powerlaw, and diskbb.
# This would be "tbabs * (powerlaw + diskbb)" in XSPEC notation
unfolded_model = LM_Model(wrap_tbabs)*(LM_Model(powerlaw)+LM_Model(wrap_diskbb))

start_params = unfolded_model.make_params(index=dict(value=-1.65,min=-3.0,max=-1.0),
                                          norm_pl=dict(value=6.5,min=1e-1,max=50.),
                                          kT=dict(value=0.25,min=0.10,max=1),
                                          norm_disk=dict(value=2e5,min=1e1,max=1e8),
                                          nH=dict(value=0.1,min=1e-3,max=10))

spectrum_fit.set_model(unfolded_model,params=start_params)

# Set the models for the time-lags
#the timing model is a composite of the tbabs, pivoting powerlaw, and reverberation
timing_model = LM_Model(wrap_tbabs_tl)*(LM_Model(pivoting_lowecut,independent_vars=['energs','freqs'])+
                                    LM_Model(reverb,independent_vars=['energs','times']))

start_params = timing_model.make_params(norm_pl2=dict(value=1,min=1e-3,max=1e4,vary=False),
                                        index=dict(value=-1.598,min=-1.8,max=-1.4,vary=False),
                                        gamma_0=dict(value=0.046,min=0,max=0.5,vary=True),
                                        gamma_slope=dict(value=-0.016,min=-0.1,max=0,vary=True),
                                        phi_0=dict(value=-2.31,min=-np.pi,max=np.pi,vary=True),
                                        phi_slope=dict(value=2.01,min=-3.,max=3.0,vary=True),
                                        nu_0=dict(value=lags_fit.freq_bounds[0],min=0.0001,max=10,vary=False),
                                        cutoff=dict(value=0.20,min=0.01,max=0.8,vary=False,expr='rev_temp'),
                                        rev_norm=dict(value=0.85,min=0,max=1e4,vary=True),
                                        rev_temp=dict(value=0.20,min=0.01,max=0.8,vary=False),
                                        rise_slope=dict(value=4,min=0,max=10,vary=False),
                                        decay_slope=dict(value=-1.46,min=-10,max=0,vary=True),
                                        peak_time=dict(value=0.01,min=0.0,max=0.2,vary=False),
                                        temp_slope=dict(value=-0.04,min=-3,max=0,vary=True),
                                        nH=dict(value=0.093,min=1e-3,max=10)
                                        )


lags_fit.set_model(timing_model,model_type="transfer")
lags_fit.set_psd_weights(psd)
lags_fit.set_params(start_params)
no_renorm_params = lags_fit.model_params
lags_fit.renorm_phases(True)

# Combining the two

Okay, now that we have two models, we want to try and share parameters between them. To do that, we're going to use ndspec's JointFit class. JointFit acts similar to a dictionary, in which it contains each FitObject next to a name, and adds functionality for performing joint fits and inference which we'll discuss later on. Importantly, you can still use the individual objects to perform individual fits like normal and you set the objects data and model individually (as we did above).

In [ ]:
from ndspec.JointFit import JointFit

bigFit = JointFit()
bigFit.add_fitobj(spectrum_fit,"time_avg_spectrum")
bigFit.add_fitobj(lags_fit,"time_lags")
bigFit.share_params(spectrum_fit,lags_fit,["index","nH"])

And then we can use this new JointFit object to fit the data in exactly the same way as we've done previously in the single observation case with a single line call.

In [ ]:
bigFit.fit_data()   

Nice! We have fitted the data, sharing two parameters between the time-averaged spectrum model and the time-lags model. Let's plot our results. We can do this by referencing the name of the observation and using the standard plotting functionality.

In [ ]:
bigFit["time_lags"].plot_model_1d()
plt.show()
bigFit["time_lags"].plot_model_2d(use_phase=True,return_plot=True)
plt.show()
bigFit["time_avg_spectrum"].plot_model(units='eeunfold')
plt.show()

### MCMC

As a small note, there is functionally no difference between using MCMC for a single power spectrum or time-averaged spectrum fit and joint fitting. There is no limit to the number of observations you can add to a single JointFit object, other than that imposed by your hardware. By default, joint fitting will evaluate all observations contained within the object, so if you want to only fit some of the observations together, you can pass a list of names of the FitObjects that you want to evaluate.

Make sure that parameters intended to be shared between models share a name, otherwise add a prefix or suffic to the parameter names so that the software can differentiate between them. nDspec does not perform any tricks to increase performance for joint fits, so having many free parameters will mean that MCMC will take the typically long time to converge for high dimensional modelling space. Consider making your model able to produce multiple data products (time-averaged spectrum and cross-spectrum for instance) to reduce the number of necessary parameters to model your data if you are having issues.